In [1]:
import numpy as np
import pandas as pd
import os
import itertools

In [2]:
from classes.model import Model
from classes.scenario import Scenario
from classes.setup import Setup
from classes.combine import Combine

setup = Setup({
    'dir_output_all_runs': 'birds',  # Optional folder name
})

# Update Torquay to have no IVT unit.
services_updates = {
    # 'PL68DH': {'Use_MSU': 0},
    # 'EX25DW': {'Nearest_MT': 'SW170QT'},  # Crazy value to show that it works.
    'TQ27AA': {'Use_IVT': 0}
}
# SCENARIO 1


# Scenario overwrites default values
scenario = Scenario({
    'name': 'drip-and-ship',
    'destination_decision_type': 0,
    'select_lsoa_method': 'nearest',
    "setup": setup,
    "run_duration": 5,  # years
    "mt_hub_postcodes":['PL68DH', 'BS105NB'],
    "services_updates":services_updates,
    "region_type_for_lsoa_selection":'ICB'
})

model = Model(scenario)

model.run()

# from classes.map import plot_map_selected_units, plot_map_catchment, plot_map_outcome
# plot_map_selected_units(
#     setup, col=scenario.region_column_for_lsoa_selection)
# # TEMPORARILY commented out for speed of running. 29th Jan 2024
# plot_map_catchment(
#     setup, col=scenario.region_column_for_lsoa_selection)
# plot_map_outcome(
#     setup, col=scenario.region_column_for_lsoa_selection,
#     outcome='mrs_shift', destination_type=scenario.destination_decision_type
# )

# SCENARIO 2

# Scenario overwrites default values
scenario = Scenario({
    'name': 'mothership',
    'destination_decision_type': 1,
    'select_lsoa_method': 'nearest',
    "setup": setup,
    "run_duration": 5,  # years
    "mt_hub_postcodes":['PL68DH', 'BS105NB'],
    "services_updates":services_updates,
    "region_type_for_lsoa_selection":'ICB'
})

model = Model(scenario)

model.run()

# Combine results across all runs
combine = Combine({'setup': setup})
combine.combine_files()


/home/anna/stroke-modelling/simpy-pathway-model/classes/units.py:346: DtypeWarning: Columns (17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_regions = pd.read_csv(path_to_file)
/home/anna/stroke-modelling/simpy-pathway-model/classes/units.py:436: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if stroke_unit in df_stroke_teams.index.values:
/home/anna/stroke-modelling/simpy-pathway-model/classes/scenario.py:381: DtypeWarning: Columns (17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_regions = pd.read_csv(path_to_file)
/home/anna/stroke-modelling/simpy-pathway-model/classes/units.py:346: DtypeWarning: Columns (17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_regions = pd.read_csv(path_to_file)
/home/anna/stroke-modelling/simpy-pathway-model/classes/units.py:436: FutureWarning: elementwise 

NameError: name 'plot_map_selected_units' is not defined

In [ ]:
setup = Setup()
setup.import_from_file('./output/birds!18/setup.yml')

In [18]:
vars(scenario)

{'name': 'mothership',
 'mt_hub_postcodes': ['PL68DH', 'BS105NB'],
 'limit_to_england': True,
 'select_lsoa_method': 'nearest',
 'region_type_for_lsoa_selection': 'ICB',
 'region_column_for_lsoa_selection': 'ICB22NM',
 'run_duration': 7200,
 'warm_up': 50,
 'destination_decision_type': 1,
 'custom_units': False,
 'probability_ivt': 1.0,
 'probability_mt': 1.0,
 'process_time_call_ambulance': (30, 30),
 'process_time_ambulance_response': (30, 30),
 'process_ambulance_on_scene_duration': (20, 20),
 'process_time_arrival_to_needle': (30, 30),
 'process_time_arrival_to_puncture': (45, 45),
 'transfer_time_delay': 30,
 'process_time_transfer_arrival_to_puncture': (60, 60),
 'services_updates': {'TQ27AA': {'Use_IVT': 0}},
 'setup': <classes.setup.Setup at 0x7ff04f32fee0>,
 'national_dict': {'hospital_services':     Postcode                               SSNAP name  Use_IVT  Use_MT  \
  0     RM70AG             Queens Hospital Romford HASU        1       1   
  1      E11BB               Roya

In [ ]:
from classes.map import Map
plot_map_selected_units(
    setup, col=scenario.region_column_for_lsoa_selection)
# TEMPORARILY commented out for speed of running. 29th Jan 2024
plot_map_catchment(
    setup, col=scenario.region_column_for_lsoa_selection)
plot_map_outcome(
    setup, col=scenario.region_column_for_lsoa_selection,
    outcome='mrs_shift', destination_type=scenario.destination_decision_type
)


In [ ]:
completed_patients_keys = model.pathway.completed_patients[0].keys()
time_cols = [x for x in completed_patients_keys if x[0:4] == 'time']
time_cols.remove('time_onset')

In [ ]:
model.results_summary_all

In [ ]:
model.results_all

In [ ]:
model.results_summary_by_admitting_unit

In [ ]:
model.results_all[np.isnan(model.results_all['time_puncture']) == False]

In [ ]:
model.results_all[np.isnan(model.results_all['time_needle']) == False]